In [7]:
from sklearn.model_selection import train_test_split

from GetData import getData


import tensorflow as tf
import numpy as np
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from GetData import getData


In [10]:
class Model:
    def __init__(self):
        self.model = tf.keras.Sequential()
        self.history={}
        self.RMSE={}
        self.config=[]
        self.activationFinction = 'relu'
        self.configs=[[2,1],[2,2,1]]
        self.num_iterations=10

    def get_RMSE(self,X1_train1, y1_train1,X1_validation, y1_validation,X_test,y_test):
        y_predict_train = self.model.predict(X1_train1)
        y_predict_val = self.model.predict(X1_validation)
        y_predict_test = self.model.predict(X_test)

        rmse_train=np.sqrt(mean_squared_error(y1_train1, y_predict_train))
        rmse_val=np.sqrt(mean_squared_error(y1_validation, y_predict_val))
        rmse_test=np.sqrt(mean_squared_error(y_test, y_predict_test))

        self.RMSE={'train':rmse_train,'validation':rmse_val,'test':rmse_test}

    def config_setup(self,num_of_features):
        activationfunction = self.activationFinction

        #clearing the model to initiate new config
        del self.model
        self.model = tf.keras.Sequential()

        self.model.add(Input(shape=(num_of_features,)))

     #   self.model.add(Input(shape=(self.config[0],)))
     #   self.model.add(Dense(self.config[1],activation='relu',input_shape=(self.config[0],)))
        for layer in range(len(self.config)):
            self.model.add(Dense(self.config[layer], activation=activationfunction))

        # Compile the model
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['mse'])

        early_stopping_callback = tf.keras.callbacks.EarlyStopping(
          monitor='val_loss',  # Monitor validation loss
          patience=5,  # Stop training if no improvement for 5 consecutive epochs
          restore_best_weights=True  # Restore model weights to the best observed during training
          )
        return early_stopping_callback

    def train_Model(self,X1_train1, y1_train1,X1_validation, y1_validation,X_test,y_test):

        num_of_features = len(X1_train1.columns)

        early_stopping_callback = self.config_setup(num_of_features)
        history = self.model.fit(X1_train1, y1_train1, epochs=50, validation_data=(X1_validation, y1_validation), callbacks=[early_stopping_callback])
        self.history = {'loss': history.history['loss'],'val_loss':history.history['val_loss']}

        self.get_RMSE(X1_train1, y1_train1,X1_validation, y1_validation,X_test,y_test)

    def save_Model(self,name):
        self.model.save(name)
        #here insert code for saving history in a separate file. model.history is lost after saving model

    #def load_history(self):

    def load_Model(self,name):
        self.model=tf.keras.models.load_model(name)

    def Model_Across_Iterations(self,X1_train1, y1_train1,X1_validation, y1_validation,X_test,y_test):
        num_iterations = self.num_iterations
        modelList=[]
        loss = []
        val_loss = []
        for iteration in range(num_iterations):
            self.train_Model(X1_train1, y1_train1,X1_validation, y1_validation,X_test,y_test)
            loss.append(self.history['loss'])
            val_loss.append(self.history['val_loss'])
            self.save_Model(f'model_{iteration}.h5')
            modelList.append(self)
        return modelList, loss, val_loss

    def Model_Across_Permutations(self,X1_train1, y1_train1,X1_validation, y1_validation,X_test,y_test):
        modelList = []
        loss = []
        val_loss = []
        configs = self.configs

        for config in configs:
            self.config = config
            modelList_per_iteration, loss_per_iteration, val_loss_per_iteration = self.Model_Across_Iterations(X1_train1, y1_train1,X1_validation, y1_validation,X_test,y_test)
            modelList.append(modelList_per_iteration)
            loss.append(loss_per_iteration)
            val_loss.append(val_loss_per_iteration)
        return modelList, loss, val_loss

    def Model_Across_DataSets(self,datasets):
      dataFrame = getData()

      modelList = []
      loss = []
      val_loss = []

      for i in datasets:
          X2=dataFrame['workableData']['FeatureDividedData'][f'X{i}']
          y=dataFrame['workableData']['y']

          X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size = 0.2, random_state=42)
          X1_train1, X1_validation, y1_train1, y1_validation = train_test_split(X_train, y_train, test_size = 0.3, random_state=42)



          modelList_per_permutation, loss_per_permutation, val_loss_per_permutation = self.Model_Across_Permutations(X1_train1, y1_train1,X1_validation, y1_validation,X_test,y_test)

          modelList.append(modelList_per_permutation)
          loss.append(loss_per_permutation)
          val_loss.append(val_loss_per_permutation)
      return modelList, loss, val_loss


In [11]:
datasets = [2,3]

model_across_datasets = Model()
model_across_datasets.num_iterations = 1
modelList, loss, val_loss = model_across_datasets.Model_Across_DataSets(datasets)


Epoch 1/50
645/645 [==============================] - 2s 2ms/step - loss: -7167124.0000 - mse: 619235377152.0000 - val_loss: -10165834.0000 - val_mse: 621481295872.0000
Epoch 2/50
645/645 [==============================] - 1s 2ms/step - loss: -10491764.0000 - mse: 619234721792.0000 - val_loss: -10643539.0000 - val_mse: 621481033728.0000
Epoch 3/50
645/645 [==============================] - 2s 3ms/step - loss: -10647589.0000 - mse: 619234459648.0000 - val_loss: -10705569.0000 - val_mse: 621480902656.0000
Epoch 4/50
645/645 [==============================] - 2s 3ms/step - loss: -10690401.0000 - mse: 619234590720.0000 - val_loss: -10738623.0000 - val_mse: 621480902656.0000
Epoch 5/50
645/645 [==============================] - 2s 3ms/step - loss: -10718097.0000 - mse: 619234525184.0000 - val_loss: -10761970.0000 - val_mse: 621480771584.0000
Epoch 6/50
645/645 [==============================] - 2s 3ms/step - loss: -10728947.0000 - mse: 619234721792.0000 - val_loss: -10777632.0000 - val_mse:

In [5]:
model = modelList[0][0][0]

print(model.RMSE)
print(model.config)
print(model.history)

print(loss)
print(val_loss[0])
print(val_loss[0][0])
print(val_loss[0][0][0])
print(val_loss[0][0][0][0])

{'train': 786916.0619914763, 'validation': 788342.017245364, 'test': 791125.1512281905}
[2, 2, 1]
{'loss': [10885662.0, 10885657.0, 10885662.0, 10885662.0, 10885660.0, 10885657.0], 'val_loss': [10918314.0, 10918314.0, 10918314.0, 10918314.0, 10918314.0, 10918314.0]}
[[[[-7151998.0, -9269923.0, -10119967.0, -10534749.0, -10538988.0, -10540401.0, -10541506.0, -10543052.0, -10545381.0, -10549129.0, -10557107.0, -10724272.0, -10761646.0, -10761644.0, -10761642.0, -10761645.0, -10761636.0]], [[10885662.0, 10885657.0, 10885662.0, 10885662.0, 10885660.0, 10885657.0]]]]
[[[-9033987.0, -9357619.0, -10559477.0, -10567152.0, -10568130.0, -10569073.0, -10570402.0, -10572336.0, -10575284.0, -10580343.0, -10595269.0, -10793927.0, -10793927.0, -10793927.0, -10793927.0, -10793927.0, -10793927.0]], [[10918314.0, 10918314.0, 10918314.0, 10918314.0, 10918314.0, 10918314.0]]]
[[-9033987.0, -9357619.0, -10559477.0, -10567152.0, -10568130.0, -10569073.0, -10570402.0, -10572336.0, -10575284.0, -10580343.0, -